# 참고자료

### Python Package
- https://mplsoccer.readthedocs.io/en/latest/

### References
- https://github.com/jakeyk11/football-data-analytics by [@_JKDS_](https://twitter.com/_JKDS_)
- https://github.com/devinpleuler/analytics-handbook by [@devinpleuler](https://twitter.com/devinpleuler)
- https://markstats.club/ by [@markrstats](https://twitter.com/markrstats)
- https://gustasam5.medium.com/ by [@GoalAnalysis](https://twitter.com/GoalAnalysis)
- [@DatoBHJ](https://twitter.com/DatoBHJ)

### Opta Event Definitions
- https://www.statsperform.com/opta-event-definitions/

# 준비

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba

from mplsoccer import Pitch, VerticalPitch, FontManager

from pathlib import Path

# 데이터 읽어오기

In [ ]:
DATAFILE = './data/Fulham_Leicster city.csv'
DATAFILE = './data/Newcastle_Arsenal.csv'

In [ ]:
df = pd.read_csv(DATAFILE)

# print(df.columns)
# print()

# for col in df.columns:
#     print("✅" + col)
#     print(df[col].value_counts().sort_values())
#     print()

In [ ]:
# 경기 및 팀 관련 기본 정보 수집

DATE = df['Date'].unique()[0]
TEAMS = [df['homeTeam'].unique()[0], df['awayTeam'].unique()[0]]

TEAMS_INFO = {}

for team in TEAMS:
    info = {}
    info['team_name_full'] = team
    info['team_name_abbrev'] = df.loc[df['Team']==team, 'teamAbbrevName'].iloc[0]
    info['team_color'] = df.loc[df['Team'] ==team, 'newestTeamColor'].iloc[0]
    info['players'] = df.loc[df['Team']==team, 'toucher'].unique().tolist()

    TEAMS_INFO[team] = info

# 아웃풋 폴더 생성
OUTPUT_DIR = f'{DATE} {TEAMS_INFO[TEAMS[0]]["team_name_abbrev"]} vs {TEAMS_INFO[TEAMS[1]]["team_name_abbrev"]}'
Path(f"./output/{OUTPUT_DIR}").mkdir(exist_ok=True)

In [ ]:
# 배경 색 / 폰트 설정
BACKGROUND_COLOR = '#1C1C1E'
TEXT_COLOR = 'white'
LINE_COLOR = 'mediumspringgreen'
TERRITORY_COLOR = 'cornflowerblue'

FONT_lIGHT = FontManager('https://github.com/google/fonts/blob/main/apache/roboto/static/Roboto-Light.ttf?raw=true')
FONT_BOLD = FontManager('https://github.com/google/fonts/blob/main/apache/roboto/static/Roboto-Bold.ttf?raw=true')

# 색 밝기 계산
def calc_L(hex_code):
    R, G, B, A = to_rgba(hex_code)
    L = np.mean([R, G, B]) / 255
    return L

# 색 어두우면 밝은 회색으로 수정
for team in TEAMS:
    if calc_L(TEAMS_INFO[team]['team_color']) < calc_L('#333333'):
        print(team, "색 수정")
        TEAMS_INFO[team]['team_color'] = '#AAAAAA'

# XG

In [ ]:
def draw_xG(df):

    df = df.sort_values("gameClock", ascending=True)
    df["gameClock"] = df["gameClock"]/60

    fig, ax = plt.subplots(figsize=(12, 8))
    fig.set_facecolor(BACKGROUND_COLOR)
    ax.set_facecolor(BACKGROUND_COLOR)

    for team in TEAMS:

        ax.step(
                df[df['Team']==team]["gameClock"], df[df['Team']==team]["xG"].cumsum(), 
                color=TEAMS_INFO[team]['team_color'],
                linewidth=2, 
                label=team, 
                )
        ax.fill_between(
                df[df['Team']==team]["gameClock"], df[df['Team']==team]["xG"].cumsum(), 
                color=TEAMS_INFO[team]['team_color'], 
                alpha=0.1, linewidth=0, 
                step='pre'
                )
        
       
        ax.spines.left.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.spines.right.set_visible(False)
        ax.spines.bottom.set_visible(False)


    plt.xlim(0, )
    plt.xticks([0, 45, 90], color=TEXT_COLOR)
    plt.tick_params(axis='x', bottom=False, top=False) 

    plt.ylim(0, )
    plt.yticks([])
    plt.tick_params(axis='y', left=False, right=False) 


    plt.xlabel("", color=TEXT_COLOR)
    plt.ylabel("xG", color=TEXT_COLOR)

    plt.legend(labelcolor=TEXT_COLOR, 
               frameon=False,
               fontsize=100, 
               prop=FONT_lIGHT.prop)
    
    
    fig.savefig(f'./output/{OUTPUT_DIR}/xG.png', dpi=300)
#     plt.show()
    plt.close('all')

In [ ]:
# xG 그리기

draw_xG(df)

# PASS NETWORK

In [ ]:
def calc_pass(df):
    # 원본 데이터프레임에서 패스 성공만 필터링
    passes_between = df[(df['playType'] == 'Pass') & (df['PsCompleted'] == 'Yes')]

    # 패스 성공 시 각 선수들의 평균 위치 및 횟수 계산
    average_locs_and_count = passes_between.groupby('passer').agg({'EventX': 'mean', 'EventY': 'mean', 'toucher': 'count'})
    average_locs_and_count.columns = ['x', 'y', 'count']

    # 선수 간 패스 성공 횟수 집계
    passes_between = passes_between.groupby(['passer', 'receiver'])['PsCompleted'].count().reset_index()
    passes_between.rename({'PsCompleted': 'pass_count'}, axis='columns', inplace=True)

    # 데이터프레임 병합
    passes_between = passes_between.merge(average_locs_and_count, left_on='passer', right_index=True)
    passes_between = passes_between.merge(average_locs_and_count, left_on='receiver', right_index=True, suffixes=['', '_end'])

    return average_locs_and_count, passes_between

In [ ]:
def draw_pass_network(df, team):   

    """패스 관련 데이터 준비"""
    average_locs_and_count, passes_between = calc_pass(df[df['Team']==team])
    
    # 선 굵기 / 노드 사이즈 설정
    MAX_LINE_WIDTH = 15
    MAX_MARKER_SIZE = 1500
    passes_between['width'] = (passes_between['pass_count'] / passes_between['pass_count'].max() * MAX_LINE_WIDTH)
    average_locs_and_count['marker_size'] = (average_locs_and_count['count'] / average_locs_and_count['count'].max() * MAX_MARKER_SIZE)

    # 선 투명도 설정
    MIN_TRANSPARENCY = 0.1
    c_transparency = passes_between['pass_count'] / passes_between['pass_count'].max()
    c_transparency = (c_transparency * (1 - MIN_TRANSPARENCY)) + MIN_TRANSPARENCY
    linecolor = np.array(to_rgba(LINE_COLOR))
    linecolor = np.tile(linecolor, (len(passes_between), 1))
    linecolor[:, 3] = c_transparency * 0.25


    """준비"""
    pitch = VerticalPitch(
                        pitch_type='opta',
                        pitch_color=BACKGROUND_COLOR,
                        line_color='#c7d5cc',
                        linewidth=0.1,
                        )

    fig, ax = pitch.grid(
                        figheight=16,
                        axis=False,
                        grid_height=0.82,
                        title_height=0.08,
                        title_space=0,
                        endnote_height=0.05,
                        endnote_space=0,
                        )

    fig.set_facecolor(BACKGROUND_COLOR)


    """그리기"""
    pitch.lines(
                passes_between['x'], passes_between['y'],
                passes_between['x_end'], passes_between['y_end'], 
                lw=passes_between['width'],
                color=linecolor, 
                zorder=1,
                ax=ax['pitch']
                )

    pitch.scatter(
                average_locs_and_count['x'], average_locs_and_count['y'],
                s=average_locs_and_count['marker_size'],
                color=TEAMS_INFO[team]['team_color'],
                edgecolors='#c7d5cc', 
                linewidth=0.5, alpha=1, 
                ax=ax['pitch']
                )

    for player, row in average_locs_and_count.iterrows():
        pitch.annotate(
                    player, 
                    xy=(row['x']-((row['marker_size']/2)**0.22), row['y']), # 선수명 아래로 미세 조정
                    c='white', 
                    va='center',
                    ha='center', 
                    size=14, 
                    fontproperties=FONT_lIGHT.prop,
                    ax=ax['pitch']
                    )

        
    """제목 / 각주"""
    ax['title'].text(
                    0.5, 0.55, 
                    s=team.upper(), 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=32,
                    fontproperties=FONT_BOLD.prop)
    ax['title'].text(
                    0.5, 0.15, 
                    s="PASS NETWORK", 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=18,
                    fontproperties=FONT_lIGHT.prop)
    ax['endnote'].text(
                    0.5, 0.5, 
                    s=DATE, 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=18,
                    fontproperties=FONT_lIGHT.prop)
    


    fig.savefig(f'./output/{OUTPUT_DIR}/PASS_NETWORK - {team}.png', dpi=300)
    # plt.show()
    plt.close('all')



In [ ]:
def draw_pass_network_pair(df):

    """준비"""
    pitch = VerticalPitch(
                        pitch_type='opta',
                        pitch_color=BACKGROUND_COLOR,
                        line_color='#c7d5cc',
                        linewidth=0.1,
                        )

    fig, ax = pitch.grid(
                        ncols=2,
                        figheight=16,
                        axis=False,
                        grid_height=0.82,
                        title_height=0.08,
                        title_space=0,
                        endnote_height=0.05,
                        endnote_space=0,
                        )

    fig.set_facecolor(BACKGROUND_COLOR)


    """팀마다 반복문"""
    for index, team in enumerate(TEAMS):

        """패스 관련 데이터 준비"""
        average_locs_and_count, passes_between = calc_pass(df[df['Team']==team])
        
        # 선 굵기 / 노드 사이즈 설정
        MAX_LINE_WIDTH = 15
        MAX_MARKER_SIZE = 1500
        passes_between['width'] = (passes_between['pass_count'] / passes_between['pass_count'].max() * MAX_LINE_WIDTH)
        average_locs_and_count['marker_size'] = (average_locs_and_count['count'] / average_locs_and_count['count'].max() * MAX_MARKER_SIZE)

        # 선 투명도 설정
        MIN_TRANSPARENCY = 0.1
        c_transparency = passes_between['pass_count'] / passes_between['pass_count'].max()
        c_transparency = (c_transparency * (1 - MIN_TRANSPARENCY)) + MIN_TRANSPARENCY
        linecolor = np.array(to_rgba(LINE_COLOR))
        linecolor = np.tile(linecolor, (len(passes_between), 1))
        linecolor[:, 3] = c_transparency * 0.25


        """그리기"""
        pitch.lines(
                    passes_between['x'], passes_between['y'],
                    passes_between['x_end'], passes_between['y_end'], 
                    lw=passes_between['width'],
                    color=linecolor, 
                    zorder=1,
                    ax=ax['pitch'][index]
                    )

        pitch.scatter(
                    average_locs_and_count['x'], average_locs_and_count['y'],
                    s=average_locs_and_count['marker_size'],
                    color=TEAMS_INFO[team]['team_color'],
                    edgecolors='#c7d5cc', 
                    linewidth=0.5, alpha=1, 
                    ax=ax['pitch'][index]
                    )

        for player, row in average_locs_and_count.iterrows():
            pitch.annotate(
                        player, 
                        xy=(row['x']-((row['marker_size']/2)**0.22), row['y']), # 선수명 아래로 미세 조정
                        c='white', 
                        va='center',
                        ha='center', 
                        size=14, 
                        fontproperties=FONT_lIGHT.prop,
                        ax=ax['pitch'][index]
                        )


        ax['pitch'][index].text(
                            50, 105, 
                            s=team.upper(), 
                            color='#c7d5cc',
                            va='center', 
                            ha='center', 
                            fontsize=24,
                            fontproperties=FONT_lIGHT.prop)
    

    """제목 / 각주"""
    ax['title'].text(
                    0.5, 0.55, 
                    s="PASS NETWORK", 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=32,
                    fontproperties=FONT_BOLD.prop)
    ax['endnote'].text(
                    0.5, 0.5, 
                    s=DATE, 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=18,
                    fontproperties=FONT_lIGHT.prop)

    fig.savefig(f'./output/{OUTPUT_DIR}/PASS_NETWORK.png', dpi=300)
    # plt.show()
    plt.close('all')


In [ ]:
# 하나씩 그리기
for team in TEAMS:
    draw_pass_network(df, team)


# 쌍으로 그리기
draw_pass_network_pair(df)

# TERRITORY MAP

In [ ]:
# playType 기준으로 공격/수비 이벤트 설정
# https://www.statsperform.com/opta-event-definitions/

ACTIONS_DICT = {
    'in possession' : [
        'Pass', 
        'BallTouch', 
        'Dispossessed', #소유권 상실
        'TakeOn',       # 돌파
        'OffsidePass',  # 오프사이드 패스
        'BlockedPass',  # 차단된 패스
        'Miss'          # 찬스 미스
        # 'AttemptSaved', # 차단된 득점 시도...?
        # 'FreeKick', 
        # 'PenaltyGoal', 
        # 'Goal', 
        ],  
    'out of possession' : [
        'Tackle', 
        'Interception', # 가로채기
        'Clearance',    # 걷어내기
        'Claim',        # 공중볼 처리
        'Save'          # 선방
        ]
    }

In [ ]:
def calc_actions(df, type):

    # 원본 데이터프레임에서 액션 이벤트만 필터링
    actions = df[df['playType'].isin(ACTIONS_DICT[type])]

    # 액션 시 각 선수들의 평균 위치 및 횟수 계산
    average_locs_and_count = actions.groupby('toucher').agg({'EventX': 'mean', 'EventY': 'mean', 'toucher': 'count'})
    average_locs_and_count.columns = ['x', 'y', 'count']

    return average_locs_and_count, actions

In [ ]:
def draw_action_territory_by_team(df, team):


    """준비"""
    pitch = VerticalPitch(
                        pitch_type='opta',
                        pitch_color=BACKGROUND_COLOR,
                        line_color='#c7d5cc',
                        linewidth=0.1,
                        )

    fig, ax = pitch.grid(
                        ncols=2,
                        figheight=16,
                        axis=False,
                        grid_height=0.82,
                        title_height=0.08,
                        title_space=0,
                        endnote_height=0.05,
                        endnote_space=0,
                        )

    fig.set_facecolor(BACKGROUND_COLOR)


    """OFFENSIVE/DEFENSIVE 나눠서"""
    for index, type in enumerate(ACTIONS_DICT):
        average_locs_and_count, actions = calc_actions(df[df['Team']==team], type)


        """선수마다 반복문"""
        for i, (player, row) in enumerate(average_locs_and_count.iterrows()):

            """데이터 준비"""
            # 해당 선수 데이터만 필터링
            df_player = actions[actions['toucher'] == player]
            # 이벤트 횟수 3회 미만이면 패스
            if len(df_player) < 3:
                continue


            """그리기"""
            hull = pitch.convexhull(df_player['EventX'], df_player['EventY'])
            
            pitch.polygon(hull, 
                        edgecolor=f'C{i}', facecolor=f'C{i}',  
                        alpha=0.2,
                        ax=ax['pitch'][index], 
                        )
            
            pitch.scatter(df_player['EventX'], df_player['EventY'], 
                        edgecolor='black', 
                        facecolor=f'C{i}', 
                        alpha=0.5, 
                        ax=ax['pitch'][index], 
                        )
            
            pitch.annotate(player, 
                        xy=(row['x'], row['y']), 
                        c='white', 
                        bbox=dict(boxstyle="Round", fc=f'C{i}', lw=0),
                        va='center',
                        ha='center', 
                        size=14, 
                        fontproperties=FONT_lIGHT.prop,
                        ax=ax['pitch'][index]
                        )

        """부제목"""
        ax['pitch'][index].text(
                            50, 105, 
                            s=type.title(), 
                            color='#c7d5cc',
                            va='center', 
                            ha='center', 
                            fontsize=24,
                            fontproperties=FONT_lIGHT.prop)
    

    """제목 / 각주"""
    ax['title'].text(
                    0.5, 0.55, 
                    s=team.upper(), 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=32,
                    fontproperties=FONT_BOLD.prop)
    ax['endnote'].text(
                    0.5, 0.5, 
                    s=DATE, 
                    color=TEXT_COLOR,
                    va='center', 
                    ha='center', 
                    fontsize=18,
                    fontproperties=FONT_lIGHT.prop)

    fig.savefig(f'./output/{OUTPUT_DIR}/TERRITORY_MAP - {team}.png', dpi=300)
    # plt.show()
    plt.close('all')


In [ ]:
def draw_action_territory_by_player(df):

    
    """OFFENSIVE/DEFENSIVE 나눠서"""
    for type in ACTIONS_DICT:

        # 팀 반복
        for team in TEAMS:
            average_locs_and_count, actions = calc_actions(df[df['Team']==team], type)

            # 선수 반복
            for i, (player, row) in enumerate(average_locs_and_count.iterrows()):

                """데이터 준비"""
                # 해당 선수 데이터만 필터링
                df_player = actions[actions['toucher'] == player]
                # 이벤트 횟수 3회 미만이면 패스
                if len(df_player) < 3:
                    continue
                
                
                """준비"""
                pitch = VerticalPitch(
                                    pitch_type='opta',
                                    pitch_color=BACKGROUND_COLOR,
                                    line_color='#c7d5cc',
                                    linewidth=0.1,
                                    )

                fig, ax = pitch.grid(
                                    figheight=16,
                                    axis=False,
                                    grid_height=0.82,
                                    title_height=0.08,
                                    title_space=0,
                                    endnote_height=0.05,
                                    endnote_space=0,
                                    )

                fig.set_facecolor(BACKGROUND_COLOR)


                """그리기"""
                hull = pitch.convexhull(df_player['EventX'], df_player['EventY'])
                
                pitch.polygon(hull, 
                            edgecolor=TERRITORY_COLOR, facecolor=TERRITORY_COLOR,  
                            alpha=0.2,
                            ax=ax['pitch'], 
                            )
                
                pitch.scatter(df_player['EventX'], df_player['EventY'], 
                            edgecolor='black', 
                            facecolor=TERRITORY_COLOR, 
                            alpha=0.5, 
                            ax=ax['pitch'], 
                            )
                
                
                """제목 / 각주"""
                ax['title'].text(
                                0.5, 0.55, 
                                s=player, 
                                color=TEXT_COLOR,
                                va='center', 
                                ha='center', 
                                fontsize=32,
                                fontproperties=FONT_BOLD.prop)
                ax['title'].text(
                                0.5, 0.15, 
                                s=type.title(), 
                                color=TEXT_COLOR,
                                va='center', 
                                ha='center', 
                                fontsize=18,
                                fontproperties=FONT_lIGHT.prop)
                ax['endnote'].text(
                                0.5, 0.5, 
                                s=DATE, 
                                color=TEXT_COLOR,
                                va='center', 
                                ha='center', 
                                fontsize=18,
                                fontproperties=FONT_lIGHT.prop)
    

                fig.savefig(f'./output/{OUTPUT_DIR}/TERRITORY_MAP - {team} - {player} ({type}).png', dpi=300)
                plt.close('all')

In [ ]:
# 팀 그리기
for team in TEAMS:
    draw_action_territory_by_team(df, team)

# 한명씩
draw_action_territory_by_player(df)